# Pytorch for Boston Classification

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import (
    Dataset, DataLoader
)

from sklearn.model_selection import train_test_split

from eleven.random import random_seeds
from eleven.data import create_boston_classification

In [2]:
# Some convenient formatting
torch.set_printoptions(precision=2, sci_mode=True)

In [3]:
device = torch.device('cpu')

In [4]:
x, y = create_boston_classification()

In [5]:
x = torch.tensor(x.values, dtype=torch.float32)
y = torch.tensor(y.values)

In [6]:
# Scikit-learn can handle any array like objects, including torch tensors
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42
)

## Creating  a Pytorch Dataset

In [7]:
class Boston(Dataset):
    """Boston Classification
    
    Pytorch datasets require two methods:
    `__len__`, returns the number of samples 
    in the dataset, and `__getitem__`, which
    returns the next sample of data given by an 
    index `idx`.
    
    Args:
        x: features
        y: labels
    """
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Boston(x={self.x.shape}, y={self.y.shape})"
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [8]:
train_data = Boston(x_train, y_train)
valid_data = Boston(x_test, y_test)

In [9]:
print(f"Train Dataset: {train_data}")
print(f"Valid Dataset: {valid_data}")

Train Dataset: Boston(x=torch.Size([339, 13]), y=torch.Size([339]))
Valid Dataset: Boston(x=torch.Size([167, 13]), y=torch.Size([167]))


## A Linear Model, Multi-Class Logistic Regression

In [10]:
class LinearModel(nn.Module):
    
    def __init__(self, input_dim, num_classes):
        super(LinearModel, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = self.fc(x)
        x = self.softmax(x)
        return x

In [11]:
# Our features have 13 dimensions. Our model will 
# learn weigths for each of those features. 
model = LinearModel(13, 3)

## DataLoaders

In [12]:
train_loader = DataLoader(train_data, batch_size=1)
valid_loader = DataLoader(valid_data, batch_size=1)

### DataLoader Usage:

We iterate over the dataloader, getting batches of size `batch_size`, and hand that to our model, getting softmax scores of size `(batch_size, num_classes)`.

In [13]:
for idx, (data, target) in enumerate(train_loader):
    softmax_scores = model(data)
    print(f"Model Output Shape: {softmax_scores.shape}")
    if idx == 0:
        break

Model Output Shape: torch.Size([1, 3])


### Repeatable Minibatches

If you would like have reproducible minibatches, you can use the `random_seeds` context manager I added in `eleven.random`:

In [14]:
with random_seeds():
    for idx, (data, target) in enumerate(train_loader):
        print(f"Softmax Scores: {model(data)}")
        if idx == 4:
            break

Softmax Scores: tensor([[-2.79e+02, 0.00e+00, -2.28e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[-1.62e+02, 0.00e+00, -1.84e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[-1.73e+02, 0.00e+00, -1.94e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[-2.95e+02, 0.00e+00, -2.48e+02]], grad_fn=<LogSoftmaxBackward>)
Softmax Scores: tensor([[-1.93e+02, 0.00e+00, -2.02e+02]], grad_fn=<LogSoftmaxBackward>)


## Training Loop

In [15]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [16]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [30]:
train_data = Boston(x_train, y_train)
valid_data = Boston(x_test, y_test)

train_loader = DataLoader(train_data, batch_size=32)
valid_loader = DataLoader(valid_data, batch_size=1)

model = LinearModel(13, 3)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)


with random_seeds():
    for epoch in range(500):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, valid_loader)

Train Epoch: 0 [0/339 (0%)]	Loss: 42.068401
Test set: Average loss: 25.6150, Accuracy: 92/167 (55%)

Train Epoch: 1 [0/339 (0%)]	Loss: 31.865679
Test set: Average loss: 22.1929, Accuracy: 83/167 (50%)

Train Epoch: 2 [0/339 (0%)]	Loss: 25.088993
Test set: Average loss: 18.5960, Accuracy: 76/167 (46%)

Train Epoch: 3 [0/339 (0%)]	Loss: 19.870724
Test set: Average loss: 14.2135, Accuracy: 80/167 (48%)

Train Epoch: 4 [0/339 (0%)]	Loss: 14.967349
Test set: Average loss: 9.8361, Accuracy: 83/167 (50%)

Train Epoch: 5 [0/339 (0%)]	Loss: 10.045133
Test set: Average loss: 6.0394, Accuracy: 90/167 (54%)

Train Epoch: 6 [0/339 (0%)]	Loss: 5.021724
Test set: Average loss: 4.7998, Accuracy: 94/167 (56%)

Train Epoch: 7 [0/339 (0%)]	Loss: 2.962472
Test set: Average loss: 4.0221, Accuracy: 100/167 (60%)

Train Epoch: 8 [0/339 (0%)]	Loss: 3.024350
Test set: Average loss: 3.8785, Accuracy: 99/167 (59%)

Train Epoch: 9 [0/339 (0%)]	Loss: 2.978963
Test set: Average loss: 3.7933, Accuracy: 100/167 (60%)

Test set: Average loss: 0.7716, Accuracy: 122/167 (73%)

Train Epoch: 83 [0/339 (0%)]	Loss: 0.459118
Test set: Average loss: 0.7682, Accuracy: 122/167 (73%)

Train Epoch: 84 [0/339 (0%)]	Loss: 0.458487
Test set: Average loss: 0.7649, Accuracy: 122/167 (73%)

Train Epoch: 85 [0/339 (0%)]	Loss: 0.457882
Test set: Average loss: 0.7617, Accuracy: 123/167 (74%)

Train Epoch: 86 [0/339 (0%)]	Loss: 0.457304
Test set: Average loss: 0.7584, Accuracy: 123/167 (74%)

Train Epoch: 87 [0/339 (0%)]	Loss: 0.456756
Test set: Average loss: 0.7553, Accuracy: 123/167 (74%)

Train Epoch: 88 [0/339 (0%)]	Loss: 0.456233
Test set: Average loss: 0.7522, Accuracy: 124/167 (74%)

Train Epoch: 89 [0/339 (0%)]	Loss: 0.455736
Test set: Average loss: 0.7491, Accuracy: 124/167 (74%)

Train Epoch: 90 [0/339 (0%)]	Loss: 0.455263
Test set: Average loss: 0.7461, Accuracy: 124/167 (74%)

Train Epoch: 91 [0/339 (0%)]	Loss: 0.454814
Test set: Average loss: 0.7431, Accuracy: 124/167 (74%)

Train Epoch: 92 [0/339 (0%)]	Loss:

Test set: Average loss: 0.6259, Accuracy: 127/167 (76%)

Train Epoch: 168 [0/339 (0%)]	Loss: 0.451301
Test set: Average loss: 0.6254, Accuracy: 127/167 (76%)

Train Epoch: 169 [0/339 (0%)]	Loss: 0.451240
Test set: Average loss: 0.6249, Accuracy: 127/167 (76%)

Train Epoch: 170 [0/339 (0%)]	Loss: 0.451174
Test set: Average loss: 0.6244, Accuracy: 128/167 (77%)

Train Epoch: 171 [0/339 (0%)]	Loss: 0.451102
Test set: Average loss: 0.6240, Accuracy: 128/167 (77%)

Train Epoch: 172 [0/339 (0%)]	Loss: 0.451025
Test set: Average loss: 0.6235, Accuracy: 128/167 (77%)

Train Epoch: 173 [0/339 (0%)]	Loss: 0.450944
Test set: Average loss: 0.6231, Accuracy: 128/167 (77%)

Train Epoch: 174 [0/339 (0%)]	Loss: 0.450858
Test set: Average loss: 0.6227, Accuracy: 128/167 (77%)

Train Epoch: 175 [0/339 (0%)]	Loss: 0.450767
Test set: Average loss: 0.6223, Accuracy: 128/167 (77%)

Train Epoch: 176 [0/339 (0%)]	Loss: 0.450672
Test set: Average loss: 0.6219, Accuracy: 128/167 (77%)

Train Epoch: 177 [0/339 (

Test set: Average loss: 0.6116, Accuracy: 130/167 (78%)

Train Epoch: 251 [0/339 (0%)]	Loss: 0.436866
Test set: Average loss: 0.6116, Accuracy: 130/167 (78%)

Train Epoch: 252 [0/339 (0%)]	Loss: 0.436655
Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 253 [0/339 (0%)]	Loss: 0.436443
Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 254 [0/339 (0%)]	Loss: 0.436232
Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 255 [0/339 (0%)]	Loss: 0.436021
Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 256 [0/339 (0%)]	Loss: 0.435810
Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 257 [0/339 (0%)]	Loss: 0.435599
Test set: Average loss: 0.6114, Accuracy: 130/167 (78%)

Train Epoch: 258 [0/339 (0%)]	Loss: 0.435390
Test set: Average loss: 0.6114, Accuracy: 130/167 (78%)

Train Epoch: 259 [0/339 (0%)]	Loss: 0.435180
Test set: Average loss: 0.6114, Accuracy: 130/167 (78%)

Train Epoch: 260 [0/339 (

Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 339 [0/339 (0%)]	Loss: 0.420134
Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 340 [0/339 (0%)]	Loss: 0.419971
Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 341 [0/339 (0%)]	Loss: 0.419809
Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 342 [0/339 (0%)]	Loss: 0.419648
Test set: Average loss: 0.6115, Accuracy: 130/167 (78%)

Train Epoch: 343 [0/339 (0%)]	Loss: 0.419486
Test set: Average loss: 0.6116, Accuracy: 130/167 (78%)

Train Epoch: 344 [0/339 (0%)]	Loss: 0.419326
Test set: Average loss: 0.6116, Accuracy: 130/167 (78%)

Train Epoch: 345 [0/339 (0%)]	Loss: 0.419166
Test set: Average loss: 0.6116, Accuracy: 130/167 (78%)

Train Epoch: 346 [0/339 (0%)]	Loss: 0.419007
Test set: Average loss: 0.6116, Accuracy: 130/167 (78%)

Train Epoch: 347 [0/339 (0%)]	Loss: 0.418848
Test set: Average loss: 0.6116, Accuracy: 130/167 (78%)

Train Epoch: 348 [0/339 (

Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 424 [0/339 (0%)]	Loss: 0.408363
Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 425 [0/339 (0%)]	Loss: 0.408248
Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 426 [0/339 (0%)]	Loss: 0.408132
Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 427 [0/339 (0%)]	Loss: 0.408018
Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 428 [0/339 (0%)]	Loss: 0.407905
Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 429 [0/339 (0%)]	Loss: 0.407790
Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 430 [0/339 (0%)]	Loss: 0.407677
Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 431 [0/339 (0%)]	Loss: 0.407565
Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 432 [0/339 (0%)]	Loss: 0.407452
Test set: Average loss: 0.6118, Accuracy: 133/167 (80%)

Train Epoch: 433 [0/339 (